In [1]:
from __future__ import division
import networkx as nx
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import json
import pickle
import datetime
import numpy as np

import sys
sys.path.append('../src')
from incident_diagnosis.incident_diagnosis import root_cause_localization, explain, optimize, get_weight_from_edge_info

/home/user/anaconda3/envs/pilot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('../data/issue_topoloies.pkl', 'rb') as f:
    Topologies = pickle.load(f)

In [3]:
all_cases = [case for case in Topologies if case['y'] == 1 and 'root_cause' in case and case['root_cause']!='All']

In [4]:
for case in all_cases:
    case['edge_info'] = {}
    case['edge_mount_info'] = {}

    for edge in case['edges_info']:
        case['edge_info'][edge['src']+'_'+edge['des']] = {}
        case['edge_info'][edge['src']+'_'+edge['des']]['FailCount'] = edge['FailCount']

        case['edge_mount_info'][edge['src']+'_'+edge['des']] = {}
        case['edge_mount_info'][edge['src']+'_'+edge['des']]['FailCount'] = edge['YesterFailCount']


In [5]:
init_clue_tag = 'FailCount'
node_clue_tags=[]
edge_clue_tags=['FailCount']
a = {}
for clue_tag in edge_clue_tags:
    a[clue_tag] = 1
for clue_tag in node_clue_tags:
    a[clue_tag] = 1
edge_backward_factor=0.3

for case in all_cases:
    case['pred'] = root_cause_localization(case, node_clue_tags, edge_clue_tags, a, get_edge_weight=get_weight_from_edge_info, edge_backward_factor=edge_backward_factor)

In [6]:
for case in all_cases:
    case['right'] = case['pred']==case['root_cause']

In [7]:
pd.Series([case['right'] for case in all_cases]).value_counts()

True     384
False     18
Name: count, dtype: int64

In [8]:
376/(376+26)

0.9353233830845771